In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.applications import DenseNet121
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.optimizers import Optimizer

# Set parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 0.001
CSV_PATH = '/Users/pillala/Documents/capstone project/messidor-2/messidor_data.csv'
SEGMENTED_FOLDER = '/Users/pillala/Documents/capstone project/messidor-2/segmented_images/'

# 1. Load CSV data and filter gradable images
data = pd.read_csv(CSV_PATH)
data = data[data['adjudicated_gradable'] == 1]
image_paths = data['id_code'].apply(lambda x: os.path.join(SEGMENTED_FOLDER, x)).values
labels = data['diagnosis'].values

# 2. One-hot encode the labels
num_classes = len(np.unique(labels))
labels = to_categorical(labels, num_classes=num_classes)

# 3. Data Generator Class
class DataGenerator(Sequence):
    def __init__(self, image_paths, labels, batch_size, img_size, shuffle=True):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return len(self.image_paths) // self.batch_size

    def __getitem__(self, index):
        batch_paths = self.image_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]
        
        images = [self.load_image(path) for path in batch_paths]
        return np.array(images), np.array(batch_labels)

    def on_epoch_end(self):
        if self.shuffle:
            temp = list(zip(self.image_paths, self.labels))
            np.random.shuffle(temp)
            self.image_paths, self.labels = zip(*temp)

    def load_image(self, path):
        img = cv2.imread(path)
        img = cv2.resize(img, self.img_size)
        return img / 255.0

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

train_generator = DataGenerator(X_train, y_train, BATCH_SIZE, IMG_SIZE)
val_generator = DataGenerator(X_val, y_val, BATCH_SIZE, IMG_SIZE)

# 4. Define the Lion Optimizer
class Lion(Optimizer):
    def __init__(self, learning_rate=0.001, beta_1=0.9, beta_2=0.99, name="Lion", **kwargs):
        super(Lion, self).__init__(name, **kwargs)

        # Handle the learning rate initialization
        self._learning_rate = self._build_learning_rate(learning_rate)
        self.beta_1 = beta_1
        self.beta_2 = beta_2

    def _build_learning_rate(self, lr):
        if isinstance(lr, (float, int)):
            return tf.Variable(lr, name="learning_rate", trainable=False)
        elif isinstance(lr, tf.keras.optimizers.schedules.LearningRateSchedule):
            return lr
        else:
            raise ValueError("learning_rate must be a float, int, or LearningRateSchedule.")

    def _create_slots(self, var_list):
        for var in var_list:
            self.add_slot(var, "m")

    def _resource_apply_dense(self, grad, var, apply_state=None):
        lr_t = tf.convert_to_tensor(self._learning_rate, var.dtype)
        m = self.get_slot(var, "m")

        new_m = self.beta_1 * m + (1 - self.beta_1) * tf.sign(grad)
        var_update = self.beta_2 * var - lr_t * new_m

        updates = [m.assign(new_m), var.assign(var_update)]
        return tf.group(*updates)

    def get_config(self):
        config = super(Lion, self).get_config()
        config.update({
            "learning_rate": self._learning_rate.numpy(),
            "beta_1": self.beta_1,
            "beta_2": self.beta_2,
        })
        return config

# 5. Define the EDenseNet Model
def build_edensenet(input_shape, num_classes):
    base_model = DenseNet121(input_shape=input_shape, include_top=False, weights='imagenet', name='edensenet')
    base_model.trainable = False  # Freeze the base model

    x = GlobalAveragePooling2D()(base_model.output)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    return model

# Initialize the model
model = build_edensenet((224, 224, 3), num_classes)

# Initialize Lion optimizer
lion_optimizer = Lion(learning_rate=LEARNING_RATE)

# Compile the model with the Lion optimizer
model.compile(optimizer=lion_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 6. Train the model
history = model.fit(train_generator, validation_data=val_generator, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

# 7. Evaluate the model
y_pred = np.argmax(model.predict(val_generator), axis=1)
y_true = np.argmax(np.concatenate([y for _, y in val_generator], axis=0), axis=1)

print(classification_report(y_true, y_pred, zero_division=1))

# 8. Confusion Matrix
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# 9. Plot Training and Validation Accuracy
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# 10. Plot Training and Validation Loss
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

ValueError: Argument `learning_rate` should be float, or an instance of LearningRateSchedule, or a callable (that takes in the current iteration value and returns the corresponding learning rate value). Received instead: learning_rate=Lion